# Testing C++ code in python
June 12, 2020

In [ ]:
import numpy as np

Steps to implement spectrum calculation
- Compute FFT
- Shift frequencies
- Compute radial average

## Save image for C++ code

In [ ]:
# ### Save single 2D image to csv for C++ code to handle
# fname='data/gen_images.npy'
# a1=np.load(fname)
# print(a1.shape)
# np.savetxt('data/images.csv',a1[0,0,:5,:5],delimiter=',',newline=',')

## Read input

In [3]:
fname='data/images.csv'
a1=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(5,5)
print(a1.shape)
a1

(5, 5)


array([[-0.85108137, -0.92862177, -0.9220354 , -0.95539463, -0.94620359],
       [-0.8610757 , -0.88159513, -0.92104948, -0.92366546, -0.88613319],
       [-0.83001596, -0.80970192, -0.89364105, -0.94259214, -0.84763706],
       [-0.84823382, -0.75111425, -0.87149537, -0.9284814 , -0.82969105],
       [-0.86204982, -0.84651887, -0.90997303, -0.9561035 , -0.8598606 ]])

## FFT

#### 1D FFT

In [28]:
y1=[(1.0*10.0*(e+1))-(1.0j*(e+1)) for e in range(6)]
# y1=[(i+1)*12.0 for i in range(6)]
print(y1)

[(10-1j), (20-2j), (30-3j), (40-4j), (50-5j), (60-6j)]


In [29]:
z1=np.fft.fft(y1)
# z1=np.fft.fftshift(np.fft.fft(y1))

for i in z1:
    print(i)
    
print(abs(z1))

(210-21j)
(-24.803847577293368+54.96152422706631j)
(-28.26794919243112+20.32050807568877j)
(-30+3j)
(-31.73205080756888-14.320508075688771j)
(-35.19615242270663-48.96152422706631j)
[211.04738804  60.29925373  34.81379037  30.14962686  34.81379037
  60.29925373]


### 2D FFT

In [33]:
y2=np.array([(10.0*(x+1)+5*y)+(0.0j*(((x+3)+y*2.0))) for y in range(5) for x in range(5)]).reshape(5,5)
# y2=a1
y2

array([[10.+0.j, 20.+0.j, 30.+0.j, 40.+0.j, 50.+0.j],
       [15.+0.j, 25.+0.j, 35.+0.j, 45.+0.j, 55.+0.j],
       [20.+0.j, 30.+0.j, 40.+0.j, 50.+0.j, 60.+0.j],
       [25.+0.j, 35.+0.j, 45.+0.j, 55.+0.j, 65.+0.j],
       [30.+0.j, 40.+0.j, 50.+0.j, 60.+0.j, 70.+0.j]])

In [34]:
z2=np.fft.fft2(y2)

In [39]:
for i in z2: 
    for j in i:
        print(np.around(j,decimals=4),end='\t')
    print('')
#     print([np.around(j,decimals=4) for j in i])


for i in z2: 
    for j in i:
        print(abs(j),end='\t')
    print('')

(1000+0j)	(-125+172.0477j)	(-125+40.615j)	(-125-40.615j)	(-125-172.0477j)	
(-62.5+86.0239j)	0j	0j	0j	-0j	
(-62.5+20.3075j)	(-0+0j)	(-0-0j)	(-0+0j)	(-0+0j)	
(-62.5-20.3075j)	(-0-0j)	(-0-0j)	(-0+0j)	(-0-0j)	
(-62.5-86.0239j)	0j	-0j	-0j	-0j	
1000.0	212.66270208800998	131.4327780297834	131.4327780297834	212.66270208800998	
106.33135104400495	1.386309034710929e-14	8.84589569552496e-15	8.84589569552496e-15	3.4776436809271494e-16	
65.71638901489167	1.1773967043451597e-14	8.88224570866061e-15	9.040356948573041e-15	8.903659738939497e-15	
65.71638901489167	8.903659738939497e-15	9.040356948573041e-15	8.88224570866061e-15	1.1773967043451597e-14	
106.33135104400495	3.4776436809271494e-16	8.84589569552496e-15	8.84589569552496e-15	1.386309034710929e-14	


## Frequency shift

In [ ]:
! cat data/images.csv

In [ ]:
freqs = np.fft.fftfreq(9, d=1./9)
print(freqs)
print(np.fft.fftshift(freqs))

freqs_2d=freqs.reshape(3,3)
print(freqs_2d)
np.fft.fftshift(freqs_2d, axes=(1,))

In [ ]:
a1=np.arange(-2,10)
print(a1)
np.fft.fftshift(a1)

In [ ]:
### C++ like code
for i in range(3):
    for j in range(3):
        print(freqs_2d[i,j],'\t',end='')
    print()
        

## Radial profile

In [ ]:
## Optimized python code
def f_radial_profile(data, center=(None,None)):
    ''' Module to compute radial profile of a 2D image '''
    y, x = np.indices((data.shape)) # Get a grid of x and y values
    
    if center[0]==None and center[1]==None:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0]) # compute centers
        
    # get radial values of every pair of points
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r.astype(np.int)
    
    # Compute histogram of r values
    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel()) 
    radialprofile = tbin / nr
    
    return radialprofile

In [ ]:
x_size,y_size=4,4
x_max,x_min=x_size-1,0
y_max,y_min=y_size-1,0

center_x=(x_max-x_min)/2.0
center_y=(y_max-y_min)/2.0
print(center_x,center_y)

In [ ]:
img_arr=np.random.randint(0,10,size=x_size*y_size).reshape(x_size,y_size)
img_arr

In [ ]:
img_arr=np.array([23, 6, 17, 35,33, 15, 26, 12,9, 21, 2, 27,10,19,3,6]).reshape(4,4)
print(img_arr)

In [ ]:
max_r=np.sqrt((x_size-center_x)**2+(y_size-center_y)**2).astype(int)
r_bins=np.zeros(max_r,dtype=int)
r_arr=np.zeros(max_r,dtype=np.float64)
r_prof=np.zeros(max_r,dtype=np.float64)

for x in range(x_min,x_size):
    for y in range(y_min,y_size):
        r=np.sqrt((x-center_x)**2+(y-center_y)**2)
        r_int=np.int(r)
        r_bins[r_int]+=1
        r_arr[r_int]+=img_arr[x,y]
        print(x,y,r,r_int,img_arr[x,y])

### Take ratios to get the radial profile
for i in np.arange(len(r_arr)):
    if r_bins[i]!=0: 
        r_prof[i]=(r_arr[i]/r_bins[i])



In [ ]:
r_bins,r_arr,r_prof

In [ ]:
f_radial_profile(img_arr)